In [1]:
!pip install openpyxl

In [2]:
!pip install --upgrade jupyter ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 2.0 MB/s eta 0:00:001.9 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.4 MB/s eta 0:00:00m eta 0:00:010:00:01


  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 4.0.9
    Uninstalling widgetsnbextension-4.0.9:
      Successfully uninstalled widgetsnbextension-4.0.9
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 8.1.1
    Uninstalling ipywidgets-8.1.1:
      Successfully uninstalled ipywidgets-8.1.1
  Attempting uninstall: jupyter
    Found existing installation: jupyter 1.0.0
    Uninstalling jupyter-1.0.0:
      Successfully uninstalled jupyter-1.0.0


In [3]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
import csv

In [8]:
csv_file ='confusion_finding.csv' 
df = pd.read_csv(csv_file)
df.head()

,Sentence,Label
0,I'm glad you brought that up.,not confused
1,That's a thought-provoking concept.,not confused
2,What’s the reason for this approach?,confused
3,I’m not sure what this term refers to.,confused
4,Are you assuming that...?,confused


In [9]:
df.Label.value_counts()

confused        421
not confused    285
Name: Label, dtype: int64

In [11]:
possible_labels = df.Label.unique()
possible_labels

array(['not confused', 'confused'], dtype=object)

In [12]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [13]:
label_dict

{'not confused': 0, 'confused': 1}

In [15]:
df['label'] = df.Label.replace(label_dict)

In [16]:
df.head()

,Sentence,Label,label
0,I'm glad you brought that up.,not confused,0
1,That's a thought-provoking concept.,not confused,0
2,What’s the reason for this approach?,confused,1
3,I’m not sure what this term refers to.,confused,1
4,Are you assuming that...?,confused,1


In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=17, 
                                                  stratify=df.label.values)

In [19]:
X_train

array([ 32, 131, 494, 459, 470, 261, 232, 231, 590, 396, 507, 319,  40,
        34, 517, 282, 453, 446, 701, 408, 297,  25,  79, 270,  15,  86,
       343, 561, 256, 475, 591, 449, 601, 229, 308, 534, 265, 359, 461,
       477, 696, 419, 700,   9, 273, 623, 504, 216, 163,  63, 651, 497,
       698,   0, 299, 126, 526, 398, 324, 320, 421, 627, 376,  44, 288,
       183, 322, 508, 412,  64, 335, 384, 217, 581, 598, 684, 551, 361,
       205, 567, 666, 212, 580, 230, 375, 523, 329, 466, 152, 577, 278,
       704, 693, 692,  92,  46, 253, 104, 170, 638, 105, 257, 635, 374,
        96, 514, 121, 587, 474, 304, 263, 195, 285, 236, 560, 610, 345,
       387, 129, 227, 276, 349,  56, 663, 214, 209,  16, 401,  13, 538,
        50,  98, 604, 124, 331,  65, 326, 109, 432, 641, 207,  97, 404,
       255, 486, 204, 356,  76, 372,  29, 520,   4,  33, 631, 485, 680,
       473, 525, 448, 274, 293, 391, 452, 266, 182, 362, 334, 493, 499,
       136, 330, 602, 313, 143, 360, 341, 177, 399, 481, 171, 29

In [20]:
df['data_type'] = ['not_set']*df.shape[0]

In [21]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [22]:
df

,Sentence,Label,label,data_type
0,I'm glad you brought that up.,not confused,0,train
1,That's a thought-provoking concept.,not confused,0,train
2,What’s the reason for this approach?,confused,1,train
3,I’m not sure what this term refers to.,confused,1,train
4,Are you assuming that...?,confused,1,train
...,...,...,...,...
701,The performance metrics will be evaluated next...,not confused,0,train
702,What are the underlying assumptions in this me...,confused,1,train
703,Can you provide a real-world scenario?,confused,1,train
704,I'm concerned about the practicality of that.,confused,1,train


In [24]:
df.groupby(['Label', 'label', 'data_type']).count()

Sentence
Label        label data_type          
confused     1     train           358
                   val              63
not confused 0     train           242
                   val              43

In [25]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [28]:
df[df.data_type=='train'].Sentence.values

array(["I'm glad you brought that up.",
       "That's a thought-provoking concept.",
       'What’s the reason for this approach?',
       'I’m not sure what this term refers to.',
       'Are you assuming that...?',
       'What additional resources would help clarify this?',
       'The upcoming conference will cover industry trends.',
       "I'm impressed by your knowledge.",
       'The project goals need to be revised.',
       "I don't get why this is important for our project.",
       "I'm confused about the purpose of this method.",
       "Can you explain this in a way that's easier to understand?",
       'What are the implications of this?',
       'Could you clarify the steps involved in this process?',
       'This is a bit overwhelming.', 'Why is this approach appropriate?',
       'Please provide your feedback on the updated proposal.',
       'Can you explain this concept in more detail?',
       "That's a great question.",
       "I'm not sure how this principle app

In [29]:
df.iloc[X_val[0]]

Sentence     How is this concept applied in real scenarios?
Label                                              confused
label                                                     1
data_type                                               val
Name: 166, dtype: object

In [30]:
df.iloc[y_val[0]]

Sentence     That's a thought-provoking concept.
Label                               not confused
label                                          0
data_type                                  train
Name: 1, dtype: object

In [31]:
X_val[0]

166

In [65]:
# from transformers import AutoTokenizer
# import torch
# import pandas as pd

# # Replace 'model_name' with the name of the pre-trained model you're using
# model_name = 'bert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Assuming you have a DataFrame named 'df'
# for index, row in df.iterrows():
#     data_type = row['data_type']
#     message = row['message']
#     label = row['label']

#     encoded_data = tokenizer.encode_plus(
#         text=message,
#         add_special_tokens=True,
#         return_attention_mask=True,
#         pad_to_max_length=True,
#         max_length=256,
#         return_tensors='pt'
#     )

#     input_ids = encoded_data['input_ids']
#     attention_mask = encoded_data['attention_mask']
    
#     #print(f"Processing row {index + 1} - Data Type: {data_type}, Message: {message}, Label: {label}")
    
#     # Here you can proceed with storing or processing 'input_ids', 'attention_mask', and 'label'
 

In [33]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/mdr614/anaconda3/envs/toxic_gru/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [34]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [35]:
len(dataset_train)

600

In [36]:
len(dataset_val)

106

In [37]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [40]:
batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [41]:
from transformers import get_linear_schedule_with_warmup

In [42]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [43]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [44]:
import numpy as np
from sklearn.metrics import f1_score

In [45]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [46]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [47]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [49]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [50]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/75 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.3522439541419347
Validation loss: 0.07990533193307263
F1 Score (Weighted): 1.0


Epoch 2:   0%|          | 0/75 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.041354295803854864
Validation loss: 0.007717172831429967
F1 Score (Weighted): 1.0


...


Epoch 3
Training loss: 0.006475128953655561
Validation loss: 0.003480867640714028
F1 Score (Weighted): 1.0


Epoch 4:   0%|          | 0/75 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.0038529438183953365
Validation loss: 0.002485727101364838
F1 Score (Weighted): 1.0


Epoch 5:   0%|          | 0/75 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.0031848940098037322
Validation loss: 0.002240343119150826
F1 Score (Weighted): 1.0


In [51]:
val_loss, predictions, true_vals = evaluate(dataloader_validation)

In [52]:
preds_flat = np.argmax(predictions, axis=1).flatten()

In [53]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [55]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [56]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [57]:
accuracy_per_class(predictions, true_vals)

Class: not confused
Accuracy: 43/43

Class: confused
Accuracy: 63/63



In [58]:
len(predictions), len(true_vals)

(106, 106)

In [59]:
from sklearn import metrics

In [60]:
print("Accuracy:",metrics.accuracy_score(preds_flat, true_vals))
print("Precision:",metrics.precision_score(preds_flat, true_vals))
print("Recall:",metrics.recall_score(preds_flat, true_vals))
print("f1 score", metrics.f1_score(preds_flat, true_vals, average='weighted'))

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
f1 score 1.0


### Testing

In [61]:
xlsx_file ='test.csv' 
#df_test = pd.read_excel(xlsx_file)
df_test = pd.read_csv(xlsx_file)
df_test.head()

,Sentence,Label
0,"I'm not sure I understand what you mean by ""qu...",1
1,That's a very interesting point.,0
2,Can you explain that again?,1
3,I see what you mean.,0
4,I'm a bit lost.,1


In [63]:
possible_labels = df_test.Label.unique()
possible_labels

array([1, 0])

In [64]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [66]:
df_test['label'] = df_test.Label.replace(label_dict)

In [68]:
encoded_test_val = tokenizer.batch_encode_plus(
    df_test.Sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

/home/mdr614/anaconda3/envs/toxic_gru/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [69]:
input_ids_test = encoded_test_val['input_ids']
attention_masks_test = encoded_test_val['attention_mask']
labels_test = torch.tensor(df_test.label.values)

In [70]:
dataset_test = TensorDataset(input_ids_test, attention_masks_test,labels_test)
batch_size = 8
dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [71]:
def test_model(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'labels':         batch[2],
                }  
        with torch.no_grad():        
            outputs = model(**inputs)
            
        logits = outputs[1]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    predictions = np.concatenate(predictions, axis=0)
    return predictions

In [72]:
pred_test = test_model(dataloader_test) 
preds_flat_test = np.argmax(pred_test, axis=1).flatten()
print(preds_flat_test)

[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]


In [73]:
import pandas as pd
pred_data = pd.DataFrame(preds_flat_test, columns = ["prediction"])
pd_cont = pd.concat([df_test["Sentence"], pred_data], axis = 1)
pd_cont.to_csv("prediction_github_comments_byPoliteBot.csv")

In [104]:
len(pred_data)

1015

In [105]:
len(X_val)

16644

In [106]:
pred_data

,prediction
0,2
1,0
2,2
3,2
4,1
...,...
1010,1
1011,1
1012,2
1013,0


In [ ]:
df_slice = df_test["message"]
df_slice.reset_index()

In [ ]:
pd_cont_rst = pd.concat([df_slice.reset_index(), pred_data], axis = 1)
pd_cont_rst.to_csv("done.csv")